<a href="https://colab.research.google.com/github/DrSunday1405/Hello-world/blob/main/Instance_based_TL(_from_20240617_TrAdaBoost).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

import copy

import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_raw_tab_A = pd.read_csv('drive/My Drive/2023_KDDI_challenge_digtal twin_dataset/train_data_A.csv')
data_raw_tab_C_train = pd.read_csv('drive/My Drive/2023_KDDI_challenge_digtal twin_dataset/train_data_C.csv')
data_raw_tab_C_test = pd.read_csv('drive/My Drive/2023_KDDI_challenge_digtal twin_dataset/test_data_C.csv')
data_raw_tab_A.head()

,time,source_name,y_true(fc),amfx1_ens3_oper-status,amfx1_ens3_phys-address,amfx1_ens3_speed,amfx1_ens3_statistics.in-octets,amfx1_ens3_statistics.in-unicast-pkts,amfx1_ens3_statistics.out-octets,amfx1_ens3_statistics.out-unicast-pkts,...,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.memory-status_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.nice_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,20210128175112-4_20210128175200,network-5gc-a,udmx1_vcpu-overload-start,0,0,0,1,1,1,1,...,0,0,-3308,0,-0.22,0.030,0,0.0,-0.030,0.24
1,20210128175529-6_20210128175610,network-5gc-a,normal,0,0,0,1,1,1,1,...,-4,0,79536,4,0.07,-0.055,0,0.0,0.095,-0.11
2,20210128175945-8_20210128180030,network-5gc-a,normal,0,0,0,1,1,1,1,...,0,0,-3756,0,0.09,0.000,0,0.0,0.090,-0.28
3,20210128180403-10_20210128180450,network-5gc-a,udmx1_ens4_interface-loss-start-70,0,0,0,1,1,1,1,...,-4,0,85520,4,0.05,0.050,0,0.0,0.050,-0.15
4,20210128180817-12_20210128180900,network-5gc-a,normal,0,0,0,1,1,1,1,...,0,0,-5044,0,0.31,-0.150,0,0.0,-0.100,-0.11


In [ ]:
# Remove the mutual columns that contain only one value from train_A and train_C
train_A_col_n1 = []
train_C_col_n1 = []
common_col_n1 = []

for col in data_raw_tab_A.columns:
  if data_raw_tab_A[col].nunique() == 1:
    train_A_col_n1.append(col)
print(len(train_A_col_n1))
print(train_A_col_n1)
print("\n")

for col in data_raw_tab_C_train.columns:
  if data_raw_tab_C_train[col].nunique() == 1:
    train_C_col_n1.append(col)
print(len(train_C_col_n1))
print(train_C_col_n1)
print("\n")

for i in train_A_col_n1:
  if i in train_C_col_n1:
    common_col_n1.append(i)
print(len(common_col_n1))
print(common_col_n1)

data_raw_tab_A = data_raw_tab_A.drop(columns=common_col_n1)
data_raw_tab_C_train = data_raw_tab_C_train.drop(columns=common_col_n1)
data_raw_tab_C_test = data_raw_tab_C_test.drop(columns=common_col_n1)

data_raw_tab_A.head()

2191
['source_name', 'amfx1_ens3_oper-status', 'amfx1_ens3_phys-address', 'amfx1_ens3_speed', 'amfx1_ens3_statistics.in-octets', 'amfx1_ens3_statistics.in-unicast-pkts', 'amfx1_ens3_statistics.out-octets', 'amfx1_ens3_statistics.out-unicast-pkts', 'amfx1_ens3_oper-status_value', 'amfx1_ens3_phys-address_value', 'amfx1_ens3_speed_value', 'amfx1_ens4_oper-status', 'amfx1_ens4_phys-address', 'amfx1_ens4_speed', 'amfx1_ens4_oper-status_value', 'amfx1_ens4_phys-address_value', 'amfx1_ens4_speed_value', 'amfx1_ens5_phys-address', 'amfx1_ens5_statistics.out-octets', 'amfx1_ens5_statistics.out-unicast-pkts', 'amfx1_ens5_oper-status_value', 'amfx1_ens5_phys-address_value', 'amfx1_ens5_speed_value', 'amfx1_lo_oper-status', 'amfx1_lo_phys-address', 'amfx1_lo_speed', 'amfx1_lo_oper-status_value', 'amfx1_lo_phys-address_value', 'amfx1_lo_speed_value', 'ausfx1_ens3_oper-status', 'ausfx1_ens3_phys-address', 'ausfx1_ens3_speed', 'ausfx1_ens3_statistics.in-octets', 'ausfx1_ens3_statistics.in-unicast-pk

,time,y_true(fc),amfx1_ens3_statistics.in-octets_value,amfx1_ens3_statistics.in-unicast-pkts_value,amfx1_ens3_statistics.out-octets_value,amfx1_ens3_statistics.out-unicast-pkts_value,amfx1_ens4_statistics.in-octets,amfx1_ens4_statistics.in-unicast-pkts,amfx1_ens4_statistics.out-octets,amfx1_ens4_statistics.out-unicast-pkts,...,upfx7_memory-stats.committed-percent_value,upfx7_memory-stats.free-number_value,upfx7_memory-stats.free-percent_value,upfx7_memory-stats.used-number_value,upfx7_memory-stats.used-percent_value,upfx7_per-core-stats.per-core-stat.idle_value,upfx7_per-core-stats.per-core-stat.io-wait_value,upfx7_per-core-stats.per-core-stat.sirq_value,upfx7_per-core-stats.per-core-stat.system_value,upfx7_per-core-stats.per-core-stat.user_value
0,20210128175112-4_20210128175200,udmx1_vcpu-overload-start,100146,1505,2550911,2254,1,1,1,1,...,0,3308,0,-3308,0,-0.22,0.030,0.0,-0.030,0.24
1,20210128175529-6_20210128175610,normal,107778,1619,2562513,2332,1,1,1,1,...,0,-79536,-4,79536,4,0.07,-0.055,0.0,0.095,-0.11
2,20210128175945-8_20210128180030,normal,102906,1547,2561033,2301,1,1,1,1,...,0,3756,0,-3756,0,0.09,0.000,0.0,0.090,-0.28
3,20210128180403-10_20210128180450,udmx1_ens4_interface-loss-start-70,45348,676,700993,732,1,1,1,1,...,-1,-85520,-4,85520,4,0.05,0.050,0.0,0.050,-0.15
4,20210128180817-12_20210128180900,normal,105734,1583,2549200,2291,1,1,1,1,...,0,5044,0,-5044,0,0.31,-0.150,0.0,-0.100,-0.11


In [ ]:
# # 要在这里划分好数据集（训练和验证），不然后面很难用“fig=px.histogram"这个函数去观察划分结果，然后我决定了，val_A 和 val_C 都必须保持各个标签的样本数是均匀的，每个标签取样20%
# # train_A, val_A = train_test_split(data_raw_tab_A, test_size=0.2, random_state=42)
# # train_C, val_C = train_test_split(data_raw_tab_C_train, test_size=0.2)   由于data_raw_tab_C_train数量太少，直接划分有一些标签没能成功划到验证集


# # Domain A
# # Split the dataset into normal and abnormal data
# normal_data = data_raw_tab_A[data_raw_tab_A['y_true(fc)'] == "normal"]
# abnormal_data = data_raw_tab_A[data_raw_tab_A['y_true(fc)'] != "normal"]

# print("Testing the data separation from noraml and abnormal")
# print(normal_data)
# print("\n")
# print(abnormal_data)

# # Change the data from data_train_C we use to train by adjusting the value: frac. When frac is 1.0, we use 100% of train_data_C
# sampled_data_train_A, sampled_data_val_A = train_test_split(normal_data, test_size=0.2, random_state=42)
# # print("测试一下")
# for label in abnormal_data['y_true(fc)'].unique():
#     label_data = abnormal_data[abnormal_data['y_true(fc)'] == label]
#     # print(label_data.shape)
#     # print(label_data['y_true(fc)'])
#     # print("\n")
#     label_train_A, label_val_A = train_test_split(label_data, test_size=0.2, random_state=42)
#     # print("看看每个标签的划分结果")
#     # print(label_val_C['y_true(fc)'])
#     # print("\n")
#     sampled_data_train_A = pd.concat([sampled_data_train_A, label_train_A])
#     sampled_data_val_A = pd.concat([sampled_data_val_A, label_val_A])
#     # print(sampled_data_val_C.shape)
#     # print("\n")

# # Reset the index
# sampled_data_train_A.reset_index(drop=True, inplace=True)
# train_A = sampled_data_train_A.sample(frac=1, random_state=42)
# sampled_data_val_A.reset_index(drop=True, inplace=True)
# val_A = sampled_data_val_A.sample(frac=1, random_state=42)



# # Domain C
# # Split the dataset into normal and abnormal data
# normal_data = data_raw_tab_C_train[data_raw_tab_C_train['y_true(fc)'] == "normal"]
# abnormal_data = data_raw_tab_C_train[data_raw_tab_C_train['y_true(fc)'] != "normal"]

# print("Testing the data separation from noraml and abnormal")
# print(normal_data)
# print("\n")
# print(abnormal_data)

# # Change the data from data_train_C we use to train by adjusting the value: frac. When frac is 1.0, we use 100% of train_data_C
# sampled_data_train_C, sampled_data_val_C = train_test_split(normal_data, test_size=0.2, random_state=42)
# # print("测试一下")
# for label in abnormal_data['y_true(fc)'].unique():
#     label_data = abnormal_data[abnormal_data['y_true(fc)'] == label]
#     # print(label_data.shape)
#     # print(label_data['y_true(fc)'])
#     # print("\n")
#     label_train_C, label_val_C = train_test_split(label_data, test_size=0.2, random_state=42)
#     # print("看看每个标签的划分结果")
#     # print(label_val_C['y_true(fc)'])
#     # print("\n")
#     sampled_data_train_C = pd.concat([sampled_data_train_C, label_train_C])
#     sampled_data_val_C = pd.concat([sampled_data_val_C, label_val_C])
#     # print(sampled_data_val_C.shape)
#     # print("\n")

# # Reset the index
# sampled_data_train_C.reset_index(drop=True, inplace=True)
# train_C = sampled_data_train_C.sample(frac=1, random_state=42)
# sampled_data_val_C.reset_index(drop=True, inplace=True)
# val_C = sampled_data_val_C.sample(frac=1, random_state=42)


In [ ]:
# fig=px.histogram(val_A,
#         x="y_true(fc)",
#         hover_data=val_A.columns,
#         title="Each label Ratio in the sample data")
# fig.show()

In [ ]:
# fig=px.histogram(val_C,
#         x="y_true(fc)",
#         hover_data=val_C.columns,
#         title="Each label Ratio in the sample data")
# fig.show()

In [ ]:
# Separate the labels and features in the dataset and transform them into proper data type

X_train_A = data_raw_tab_A.iloc[:,2:].values
y_train_A = data_raw_tab_A.iloc[:,1].values


X_train_C = data_raw_tab_C_train.iloc[:,2:].values
y_train_C = data_raw_tab_C_train.iloc[:,1].values
# X_val_C = val_C.iloc[:,2:]
# y_val_C = val_C.iloc[:,1]
X_test_C = data_raw_tab_C_test.iloc[:,2:].values
y_test_C = data_raw_tab_C_test.iloc[:,1].values


print(X_train_C)
print("\n")
print(type(X_train_C))
print(y_test_C.shape)


[[ 5.577200e+04  8.340000e+02  1.284770e+06 ...  0.000000e+00
   4.500000e-02 -4.500000e-02]
 [ 5.410000e+04  8.100000e+02  1.275859e+06 ...  0.000000e+00
   1.000000e-02  8.000000e-02]
 [ 1.884810e+05  9.700000e+02  1.314191e+06 ...  0.000000e+00
   1.000000e-01  4.160000e-17]
 ...
 [ 5.674200e+04  9.930000e+02  1.100393e+06 ...  0.000000e+00
  -1.900000e-01  2.100000e-01]
 [ 5.682000e+04  9.960000e+02  1.096317e+06 ...  0.000000e+00
   7.500000e-02  5.000000e-02]
 [ 3.522200e+04  6.650000e+02  2.525500e+05 ...  0.000000e+00
   1.000000e-01 -1.600000e-01]]


<class 'numpy.ndarray'>
(873,)


In [ ]:
# Encode the labels

encoder = preprocessing.LabelEncoder()
y_train_A = encoder.fit_transform(y_train_A)
y_train_C = encoder.fit_transform(y_train_C)
# y_val_C = encoder.fit_transform(y_val_C)
y_test_C = encoder.fit_transform(y_test_C)


print("After encoding the label:")
print(y_train_A)
print(type(y_train_A))
print(len(y_train_A))
print("\n")

print("检查输出")

print(y_test_C.shape)
print(y_test_C)

label_decoded_dict = dict(zip(range(len(encoder.classes_)), encoder.classes_))



After encoding the label:
[15 10 10 ... 11 10 10]
<class 'numpy.ndarray'>
3645


检查输出
(873,)
[10 10 10  1 10 10 10 14 10 10 10 10  4 10  9 10  5  6 10 10  8 10 10 10
 10 15 12 10 10 10 10  0 11 10 10 13 10 10 10 10 10  3 10  7  2 10 10 10
 10 10 10 10 10 10 10 10 10 10 10  8 10 10  7 10 10  6 10  5 12 10 10 10
 14  2 10 10 10 10  0 10 11  9 10  4 15 13  1 10 10 10  3 10  5 10 10  8
 10 12 14 10 10 10  3 10 10 10 10 10  0 11 10 10  6 10 10 10 10  2 10  4
 10 13  7 10 10 10  9 10 10 10 10 10 10  1 10 10 10 15 10  3 10 10 10  1
 10  6 10  9 10  0 10 10  7 10 10 10 10 10 10 10 15 11  8 10 10 10 10 10
 14 10 10  5 10 10 12 10  2 10 13 10 10 10 10  4  7 10 10 10  1 10 10  2
  5 15 10 10 10 10 10  9  4 12 10 10 14  8 10 10 10 10 10 13 10 10 10 10
 10 10 10 11 10 10  3 10 10 10 10  0  6 10  3 10  2 10 10 10 10 10 10 10
 10  8 10 10  0 10 10 10 10 10 15 11 10 10 10 13 10 10 10 14 10 10 10  6
  4 10  1 10 10  5  7 10 12 10 10  9  2 10 10  9  0 13  8  6 10 10  1  5
 10 15 10 10 10 10 10 10 10 10 

In [ ]:
# row_A = 4
# weights_A = np.ones([row_A, 1]) / row_A
# print(weights_A)
# print(type(weights_A))
# print("\n")

# p = np.asarray(weights_A / np.sum(weights_A), order='C')
# print(p)
# print(type(p))
# print("\n")

# print(p[:, 0])

In [ ]:
# 正式的 TrAdaBoost 算法

def calculate_P(weights):   # 归一化权重
  # print("debug1")
  total = np.sum(weights)
  return np.asarray(weights / total, order='C')     # order = "C" 是指按行存储，影响内存布局，对计算倒没有啥影响

def train_classify(trans_data, trans_label, test_data, P):

  # clf = CatBoostClassifier(iterations=30, depth=3, learning_rate=0.1, verbose=0)
  clf = CatBoostClassifier(iterations=15, depth=6, learning_rate=0.1, loss_function='MultiClass')  # 换一个参数试试
  # sample_weight = P[:, 0].flatten()
  # sample_weight = np.squeeze(P[:, 0])
  # print(sample_weight.shape)
  # print("debug3")
  clf.fit(trans_data, trans_label, sample_weight=P[:, 0])
  # 就是在这里 Catboost 函数自动考虑了每个样本的权重，来进行训练（太棒了，catboost的内置参数帮我解决了我的困惑，比神经网络要方便）
  # 这个 P[:, 0] 可能是错的，你仔细 debug一下，我感觉 P 本来是数组    我周一晚试了一下，即便 P 是一维数组，这样写也能正常运行不会报错，没关系，这里都没有问题
  # print("debug4")
  return clf.predict(test_data[:, 0])

def calculate_error_rate(label_R, label_H, weight):     # 这个计算 error-rate 的函数该不会有问题吧，源代码是对于二分类问题而已的，我这样相减的意义是什么
  # print("debug4")
  total = np.sum(weight)
  misclassify = np.sum(np.abs(label_R - label_H)) / len(label_H)
  return np.sum(weight[:, 0] / total * np.abs(label_R - label_H)), misclassify
  # 这里就是计算目标域伊普西龙的公式, 不过那个 misclassify 是干嘛的啊？（CSDN的代码没用到这个 misclassify，应该不是必需的）



def TrAdaBoost(X_source, y_source, X_target, y_target, N = 20):
  trans_data = np.concatenate((X_source, X_target), axis=0)
  trans_label = np.concatenate((y_source, y_target), axis=0)   # axis=0 是沿着行拼接

  row_A = X_source.shape[0]
  row_C = X_target.shape[0]  # shape[0]是获取行数

  # Initialize the weights
  weights_A = np.ones([row_A, 1]) / row_A
  weights_C = np.ones([row_C, 1]) / row_C
  print("A的权重初始化：{}".format(weights_A))
  print("C的权重初始化：{}".format(weights_C))
  weights = np.concatenate((weights_A, weights_C), axis=0)   # weights 是一个列向量

  bata = 1 / (1 + np.sqrt(2 * np.log(row_A / N)))     # 这是源域的 belta，是个不变的常量

  # Save prediction labels and bata_t
  bata_T = np.zeros([1, N])     # 每次迭代目标域的 bata都要记录下来，就是一个数值，所以最后能组成一个一维向量
  result_label = np.ones([row_A + row_C, N])


  print('params initial finished.')
  print('='*60)

  # 到这里为止都没有问题，前面的都是基本的初始化，不用改

  # trans_data = np.asarray(trans_data, order='C')
  # trans_label = np.asarray(trans_label, order='C')    # 将 np 向量转化为 np 数组？还是说为了打乱顺序？  GPT说它确实多余

  error_rate_list = []
  misclassify_list = []     # 这两个不是必须的，有的代码也没有

  print("Begin training\n")
  t0 = time.time()
  print(type(weights))
  print(weights.shape)

  for i in range(N):
    # print("问题1")
    P = calculate_P(weights)   # P 是向量吧，不是矩阵 （每次循环开始前要归一化刚刚更新的权重）
    # print("问题2")
    result_label[:, i] = train_classify(trans_data, trans_label, trans_data, P)
    # 训练玩之后把训练数据又放回去测试，是正确的   （但我看github上面的代码是将 test_data与 train_data 合并，然后放进去测试）  （这个点有可能是一个问题点）

    error_rate, misclassify = calculate_error_rate(y_target, result_label[row_A:row_A + row_C, i], weights[row_A:row_A + row_C, :])
    # 根据源论文的公式，这里没错，确实不是P，而是归一化前的weight值


    print("错误率是：{}".format(error_rate))

    if error_rate > 0.5:
      error_rate = 0.5   # 先这样设置，别搞太复杂

    elif error_rate <= 1e-10:   # Avoiding overfitting
      N = i
      break
    error_rate_list.append(error_rate)
    misclassify_list.append(misclassify)  # 这两个不是必需的

    bata_T[0, i] = error_rate / (1 - error_rate)   # 虽然 bata_T 是个一维数组，但这样写也行
    print("问题5")
    print('Iter {}-th result :'.format(i))
    print('error rate :', error_rate, '|| bata_T :', error_rate / (1 - error_rate))
    print('-'*60)
    # 到这里都没问题，虽然有些多余的步骤，但整体是对的，基本没有错的代码，接下来是权重更新

   # 以下这两个 for 循环 会很耗时间，要死了要死了

    # Changing the data weights of same-distribution training data
    for j in range(row_C):
      weights[row_A + j] = weights[row_A + j] * np.power(bata_T[0, i], (-np.abs(result_label[row_A + j, i] - y_target[j])))
        # Changing the data weights of diff-distribution training data
    for j in range(row_A):
      weights[j] = weights[j] * np.power(bata, np.abs(result_label[j, i] - y_source[j]))

    # 权重更新这里应该也是没问题的，你记住，weight是一个一维数组就可以了，这个数组每次迭代都更新，到这里一次循环已经结束了
    print(weights)
    print(weights.shape)


  t1 = time.time()
  print("The training time is: {}".format(t1-t0))

  print("Final weight is:\n")
  print(weights)   # 直接用这个权重放进 catboost，就完事了


  return calculate_P(weights)




In [ ]:
# 训练 TrAdaBoost 模型
N = 15  # 迭代次数
Training_weight_P = TrAdaBoost(X_train_A, y_train_A, X_train_C, y_train_C, N)


# 我建议把测试集的东西放到 TrAdaboost 函数之外

# After training, use the trained model to predict the test data
# predict = np.zeros([row_T])
# result_label_test = train_classify(trans_data, trans_label, test, np.ones([row_A + row_C, 1]) / (row_A + row_C))

# 它为什么输入这个权重进去呢？该训练好的 weight，它直接又重新均摊，肯定有问题


# 你可以回忆一下你以前写的 catboost 代码，你看看 catboost输出的是什么，，你直接用你以前的代码就可以了！我已经得到我想要的了，就是那个权重，接下来就用这个权重像以前那样运行catboost，得到测试结果
  # for i in range(row_T):
  #   left = np.sum(result_label_test[int(np.floor(N / 2)):N] * np.log(1 / bata_T[0, int(np.floor(N / 2)):N]))
  #   right = 0.5 * np.sum(np.log(1 / bata_T[0, int(np.floor(N / 2)):N]))
  #   if left >= right:
  #     predict[i] = 1
  #   else:
  #     predict[i] = 0

  # 这个很有问题，我这个都不是二分类问题来着       真的太难了，好想死啊，就是这里最头疼，github那帮脑残全都是解决二分类问题，我TM需要你教怎么解决二分类问题

print("TrAdaBoost is done")
print('='*60)
# print('The prediction labels of test data are :')
# print(test_predict)



A的权重初始化：[[0.00027435]
 [0.00027435]
 [0.00027435]
 ...
 [0.00027435]
 [0.00027435]
 [0.00027435]]
C的权重初始化：[[0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391]
 [0.00217391

In [ ]:
# 没有用 TrAdaBoost 筛选出来的weight 的时候


# Final_clf = CatBoostClassifier(iterations=30, depth=3, learning_rate=0.1, verbose=0)
Final_clf = CatBoostClassifier(iterations=15, depth=6, learning_rate=0.1, loss_function='MultiClass')  # 换一个参数试试
trans_data = np.concatenate((X_train_A, X_train_C), axis=0)
trans_label = np.concatenate((y_train_A, y_train_C), axis=0)

Final_clf.fit(trans_data, trans_label)
test_predict = Final_clf.predict(X_test_C)



test_accuracy = accuracy_score(y_test_C, test_predict)

print(f"Accuracy on validation data in Domain A: {test_accuracy:.2f}")
print("\n")

print("The confusion matrix is:")
print(confusion_matrix(y_test_C, test_predict))
print("\n")

print("The overall f1 score is:")
F2 = f1_score(y_test_C, test_predict, average="micro")
print(F2)
print("\n")

print("The f1 score of each label is:")
F1 = f1_score(y_test_C, test_predict, average=None)

for label_code, f1 in enumerate(F1):
  original_label = label_decoded_dict[label_code]
  print(f"{original_label}: {f1}")
print("\n")

0:	learn: 2.1662132	total: 2.86s	remaining: 40.1s
1:	learn: 1.7811508	total: 5.53s	remaining: 35.9s
2:	learn: 1.5422788	total: 8.15s	remaining: 32.6s
3:	learn: 1.3842199	total: 10.9s	remaining: 30.1s
4:	learn: 1.2651665	total: 15.2s	remaining: 30.4s
5:	learn: 1.1581917	total: 17.8s	remaining: 26.7s
6:	learn: 1.0619795	total: 20.3s	remaining: 23.2s
7:	learn: 0.9750252	total: 22.9s	remaining: 20.1s
8:	learn: 0.9043784	total: 25.8s	remaining: 17.2s
9:	learn: 0.8408110	total: 30s	remaining: 15s
10:	learn: 0.7885095	total: 32.6s	remaining: 11.9s
11:	learn: 0.7433798	total: 35.2s	remaining: 8.8s
12:	learn: 0.7030511	total: 37.8s	remaining: 5.81s
13:	learn: 0.6619183	total: 40.7s	remaining: 2.9s
14:	learn: 0.6229160	total: 45.8s	remaining: 0us
Accuracy on validation data in Domain A: 0.90


The confusion matrix is:
[[  0   0   0   0   0   0   0   0   0   0  19   0   0   0   0   0]
 [  0  19   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  16   0   0   0   0   0   0   0   3 

In [ ]:
# 有用 TrAdaBoost 筛选出来的weight 的时候

# Final_clf_weight = CatBoostClassifier(iterations=30, depth=3, learning_rate=0.1, verbose=0)
Final_clf_weight = CatBoostClassifier(iterations=15, depth=6, learning_rate=0.1, loss_function='MultiClass')  # 换一个参数试试
trans_data = np.concatenate((X_train_A, X_train_C), axis=0)
trans_label = np.concatenate((y_train_A, y_train_C), axis=0)

Final_clf_weight.fit(trans_data, trans_label, sample_weight=Training_weight_P[:, 0])
test_predict_weight = Final_clf_weight.predict(X_test_C)



test_accuracy = accuracy_score(y_test_C, test_predict_weight)

print(f"Accuracy on validation data in Domain A: {test_accuracy:.2f}")
print("\n")

print("The confusion matrix is:")
print(confusion_matrix(y_test_C, test_predict_weight))
print("\n")

print("The overall f1 score is:")
F2 = f1_score(y_test_C, test_predict_weight, average="micro")
print(F2)
print("\n")

print("The f1 score of each label is:")
F1 = f1_score(y_test_C, test_predict_weight, average=None)

for label_code, f1 in enumerate(F1):
  original_label = label_decoded_dict[label_code]
  print(f"{original_label}: {f1}")
print("\n")






# # 解码预测标签
# predict_labels = encoder.inverse_transform(test_predict.astype(int))

# # 输出预测结果
# print("预测标签：")
# print(predict_labels)

0:	learn: 1.8210777	total: 2.83s	remaining: 39.6s
1:	learn: 1.5296810	total: 6.56s	remaining: 42.6s
2:	learn: 1.3221340	total: 10.3s	remaining: 41.2s
3:	learn: 1.1635596	total: 14.5s	remaining: 39.9s
4:	learn: 1.0311880	total: 18.6s	remaining: 37.3s
5:	learn: 0.9260402	total: 22.7s	remaining: 34.1s
6:	learn: 0.8396870	total: 26.9s	remaining: 30.7s
7:	learn: 0.7671822	total: 30s	remaining: 26.2s
8:	learn: 0.7056899	total: 32.5s	remaining: 21.7s
9:	learn: 0.6488251	total: 35.1s	remaining: 17.6s
10:	learn: 0.6033171	total: 37.7s	remaining: 13.7s
11:	learn: 0.5625947	total: 41.8s	remaining: 10.4s
12:	learn: 0.5217407	total: 44.8s	remaining: 6.9s
13:	learn: 0.4895404	total: 47.4s	remaining: 3.39s
14:	learn: 0.4600970	total: 50s	remaining: 0us
Accuracy on validation data in Domain A: 0.80


The confusion matrix is:
[[  3   0   0   0   0   5   0   0   0   0   8   3   0   0   0   0]
 [  0  17   1   1   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  11   7   1   0   0   0   0   0   0   0